In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import requests

# from fpdf import FPDF
import pdfkit as pdfkit



In [2]:
df = pd.read_csv("./input/books.csv", error_bad_lines=False)


In [3]:
df.dtypes

bookID                  int64
title                  object
authors                object
average_rating         object
isbn                   object
isbn13                 object
language_code          object
# num_pages            object
ratings_count          object
text_reviews_count     object
Unnamed: 10           float64
Unnamed: 11           float64
Unnamed: 12           float64
dtype: object

In [4]:
df = df.drop(['bookID','isbn','language_code','text_reviews_count','ratings_count','Unnamed: 10','Unnamed: 11','Unnamed: 12'], axis=1)
df = df.rename(columns = {'# num_pages':'num_pages'})
df.head(10)

,title,authors,average_rating,isbn13,num_pages
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,9780439785969,652
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,9780439358071,870
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,9780439554930,320
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,9780439554893,352
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,9780439655484,435
5,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,9780439827607,3342
6,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,4.38,9780345453747,815
7,The Ultimate Hitchhiker's Guide (Hitchhiker's ...,Douglas Adams,4.38,9780517149256,815
8,A Short History of Nearly Everything,Bill Bryson-William Roberts,4.20,9780767908184,544
9,Bill Bryson's African Diary,Bill Bryson,3.43,9780767915069,55


In [5]:
def githubRequestAuthorized(isbn):
    authToken = os.getenv("GOOGLE_BOOKS_API_TOKEN")
    if not authToken:
        raise ValueError("NECESITAS UN TOKEN")
    url = "https://www.googleapis.com/books/v1/volumes?q=isbn:{}&key={}".format(isbn,authToken)
    #print("Requesting authorized")
    res = requests.get(url)
    data = res.json()
    return data

In [6]:
book = githubRequestAuthorized('9780439785969')

In [9]:
book['items'][0]['volumeInfo']['categories'][0]

'Juvenile Fiction'

In [10]:
for i in range(len(df)) : 
    isbn = df.loc[i, "isbn13"]
    book = githubRequestAuthorized(isbn)
    try:
        df.loc[i, 'Genre'] = book['items'][0]['volumeInfo']['categories'][0]
        
    except:
        df.loc[i, 'Genre'] = 'Others'


In [11]:
df.head(100)

,title,authors,average_rating,isbn13,num_pages,Genre
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,9780439785969,652,Juvenile Fiction
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,9780439358071,870,Juvenile Fiction
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,9780439554930,320,Juvenile Fiction
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,9780439554893,352,Juvenile Fiction
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,9780439655484,435,Juvenile Fiction
...,...,...,...,...,...,...
95,Henry Miller on Writing,Henry Miller-Thomas H. Moore,4.23,9780811201124,217,Literary Criticism
96,Quiet Days in Clichy,Henry Miller,3.69,9780802130167,154,Fiction
97,Tropic of Cancer,Henry Miller-Jiří Níl,3.69,9780802131782,318,Fiction
98,Tropic of Capricorn,Henry Miller,3.82,9780802151827,348,Fiction


In [12]:
df.to_csv(r'./input/booksWithGenre.csv')

In [90]:
df = pd.read_csv('./input/booksWithGenre.csv', index_col=0)

In [91]:
df

,title,authors,average_rating,isbn13,num_pages,Genre
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,9.780440e+12,652,Juvenile Fiction
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,9.780439e+12,870,Juvenile Fiction
2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,9.780440e+12,320,Juvenile Fiction
3,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,9.780440e+12,352,Juvenile Fiction
4,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,9.780440e+12,435,Juvenile Fiction
...,...,...,...,...,...,...
1027,Sherlock Holmes: The Unauthorized Biography,Nick Rennison,4.26,9.780871e+12,304,Others
1028,The Adventures of Sherlock Holmes,Arthur Conan Doyle-Eoin Colfer,4.31,9.780440e+12,336,Others
1029,The Mysteries of Sherlock Holmes,Arthur Conan Doyle-Paul Bachem,4.18,9.780448e+12,218,Others
1030,The Complete Adventures and Memoirs of Sherloc...,Arthur Conan Doyle,4.33,9.780517e+12,334,Others


In [92]:
dfgb = df.groupby(['Genre']).count()

In [93]:
dfsorted = dfgb.sort_values(['title'], ascending=[False]).head(20)

In [94]:
display(dfsorted[['title']])

,title
Genre,
Fiction,274
Others,227
Drama,66
Biography & Autobiography,65
History,34
Literary Criticism,33
Business & Economics,30
Juvenile Fiction,27
Philosophy,23


In [95]:
selection = df.loc[df['Genre'] == 'Fiction'].sort_values(['average_rating'], ascending=[False]).head(10)


In [96]:
selection

,title,authors,average_rating,isbn13,num_pages,Genre
773,Novels 1901–1902: The Sacred Fount / The Wings...,Henry James-Leo Bersani,4.68,9.781931e+12,713,Fiction
644,The Complete Novels,Jane Austen-Hugh Thomson,4.55,9.780517e+12,1103,Fiction
18,The Lord of the Rings Box Set,J.R.R. Tolkien,4.49,9.780618e+12,1223,Fiction
379,Enthusiasm and Divine Madness,Josef Pieper-Richard Winston-Clara Winston,4.48,9.781890e+12,125,Fiction
604,Later Novels and Other Writings: The Lady in t...,Raymond Chandler-Frank MacShane,4.47,9.781883e+12,1076,Fiction
606,The Lady in the Lake The Little Sister The L...,Raymond Chandler-Tom Hiney,4.45,9.780375e+12,1016,Fiction
363,Gates of Fire: An Epic Novel of the Battle of ...,Steven Pressfield,4.41,9.780553e+12,392,Fiction
854,Collected Tales Sketches Speeches & Essays ...,Mark Twain-Louis J. Budd,4.39,9.780940e+12,1050,Fiction
189,Reinventing the Enemy's Language: Contemporary...,Joy Harjo-Gloria Bird-Beth Cuthand-Valerie Mar...,4.39,9.780393e+12,576,Fiction
6,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,4.38,9.780345e+12,815,Fiction


In [98]:
finaldf = selection.drop(['isbn13'], axis=1)


In [99]:
finaldf

,title,authors,average_rating,num_pages,Genre
773,Novels 1901–1902: The Sacred Fount / The Wings...,Henry James-Leo Bersani,4.68,713,Fiction
644,The Complete Novels,Jane Austen-Hugh Thomson,4.55,1103,Fiction
18,The Lord of the Rings Box Set,J.R.R. Tolkien,4.49,1223,Fiction
379,Enthusiasm and Divine Madness,Josef Pieper-Richard Winston-Clara Winston,4.48,125,Fiction
604,Later Novels and Other Writings: The Lady in t...,Raymond Chandler-Frank MacShane,4.47,1076,Fiction
606,The Lady in the Lake The Little Sister The L...,Raymond Chandler-Tom Hiney,4.45,1016,Fiction
363,Gates of Fire: An Epic Novel of the Battle of ...,Steven Pressfield,4.41,392,Fiction
854,Collected Tales Sketches Speeches & Essays ...,Mark Twain-Louis J. Budd,4.39,1050,Fiction
189,Reinventing the Enemy's Language: Contemporary...,Joy Harjo-Gloria Bird-Beth Cuthand-Valerie Mar...,4.39,576,Fiction
6,The Ultimate Hitchhiker's Guide to the Galaxy,Douglas Adams,4.38,815,Fiction


In [37]:
finaldf.to_csv("./output/recommendation.csv")